In [10]:
from atexit import register
from scipy.stats import beta

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd



df1 = pd.read_csv('../res/forc_baseline.csv')
df1['method'] = 'constant'


model_name = 'Nbeats'
df2 = pd.read_csv(f'../res/{model_name}/{model_name}_base_week_pf_42test.csv')
df2['method'] = f'{model_name}_week_pf'
# df3 = pd.read_csv(f'../res/{model_name}/{model_name}_base_week_42test.csv')
# df3['method'] = f'{model_name}_base_week'

# df4 = pd.read_csv(f'../res/{model_name}/{model_name}_base_pf_42test.csv')
# df4['method'] = f'{model_name}_base_pf'
# df4= pd.read_csv('../res/gru/gru_base_holi_pf_42test.csv')
# df4['method'] = 'gru_base_holi_pf'
# df3 = pd.read_csv('../res/tcn/tcn_base_stage_holi_pf_42test.csv')
# df3['method'] = 'tcn'
# df4 = pd.read_csv('../res/Nbeats/Nbeats_base_stage_holi_pf_42test.csv')
# df4['method'] = 'Nbeats'
# df5 = pd.read_csv('itransformer_base_stage_holi_pf_ol_42test.csv')
# df5['method'] = 'itransformer'
#
df6 = pd.read_csv('../res/ensemble_model_with_intervals.csv')
df6['method'] = 'ensemble'




df = pd.concat([df1,df2], ignore_index=True)
df = df.loc[(df['mode'] == 'train') | (df['mode'].isna())| (df['mode'] == 'train_seed42') ]

df['point'] = df['point'].mask(df['point'] <= 0, 0.1)  # 仅替换 ≤0 的值


df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['original_date'] = df['date'] - pd.to_timedelta(7 * df['week_ahead'], unit='d')
df = df[df['year'] >2006]
df['squared_error'] = (df['point'] - df['true'])**2
rmse_df = df.groupby(['week_ahead'])['squared_error'].mean().reset_index()
rmse_df['RMSE'] = np.sqrt(rmse_df['squared_error'])
rmse_df = rmse_df.drop(columns=['squared_error'])

In [11]:
dft = df[['date', 'true']].drop_duplicates().sort_values('date')

In [12]:
import pandas as pd
import matplotlib.pyplot as plt  
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import tkinter as tk
from tkinter import ttk
from datetime import datetime

class WeeklyDataVisualizer:
    def __init__(self, root, df):
        self.root = root
        self.df = df
        self.df['date'] = pd.to_datetime(self.df['date'])
        self.df['original_date'] = pd.to_datetime(self.df['original_date'])
        
        # 提取年份和周数信息
        self.df['year'] = self.df['original_date'].dt.isocalendar().year
        self.df['week'] = self.df['original_date'].dt.isocalendar().week
        
        # 获取所有可用的年份和周数
        self.available_years = sorted(self.df['year'].unique())
        self.year_week_map = {}
        
        for year in self.available_years:
            self.year_week_map[year] = sorted(self.df[self.df['year'] == year]['week'].unique())
        self.setup_ui()
    
    def setup_ui(self):
        self.root.title("周数据可视化工具")
        
        # 控制面板框架
        control_frame = ttk.Frame(self.root, padding="10")
        control_frame.pack(fill=tk.X)
        
        # 年份选择器
        ttk.Label(control_frame, text="选择年份:").grid(row=0, column=0, sticky=tk.W)
        self.year_var = tk.IntVar()
        self.year_combobox = ttk.Combobox(control_frame, textvariable=self.year_var, 
                                         values=self.available_years)
        self.year_combobox.grid(row=0, column=1, sticky=tk.W)
        self.year_combobox.bind("<<ComboboxSelected>>", self.update_week_options)
        if self.available_years:
            self.year_combobox.current(0)
        
        # 周数选择器
        ttk.Label(control_frame, text="选择周数:").grid(row=0, column=2, sticky=tk.W)
        self.week_var = tk.IntVar()
        self.week_combobox = ttk.Combobox(control_frame, textvariable=self.week_var)
        self.week_combobox.grid(row=0, column=3, sticky=tk.W)
        self.update_week_options()
        
        # 范围选择
        ttk.Label(control_frame, text="前后范围(周):").grid(row=0, column=4, sticky=tk.W)
        self.range_var = tk.IntVar(value=20)
        ttk.Spinbox(control_frame, from_=1, to=52, textvariable=self.range_var, width=5).grid(row=0, column=5, sticky=tk.W)
        
        # 更新按钮
        ttk.Button(control_frame, text="更新图表", command=self.update_plot).grid(row=0, column=6, padx=10)
        
        # 图表框架
        self.figure = plt.Figure(figsize=(10, 5), dpi=100)
        self.canvas = FigureCanvasTkAgg(self.figure, master=self.root)
        self.canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)
        
        # 初始绘图
        self.update_plot()
    
    def update_week_options(self, event=None):
        selected_year = self.year_var.get()
        if selected_year in self.year_week_map:
            weeks = self.year_week_map[selected_year]
            self.week_combobox['values'] = weeks
            if weeks:
                self.week_combobox.current(0)
    
    def get_start_date(self, year, week):
        date_str = f"{year}-W{week:02d}-0"
        return datetime.strptime(date_str, "%Y-W%W-%w").date()
    
    def update_plot(self):
        selected_year = self.year_var.get()
        selected_week = self.week_var.get()
        range_weeks = self.range_var.get()
        
        try:
            # 获取所选年份和周对应的起始日期(该周周一)
            start_date = self.get_start_date(selected_year, selected_week)
            start_date = pd.to_datetime(start_date)
            
            # 计算前后范围的日期
            end_date = start_date + pd.DateOffset(weeks=range_weeks)
            start_range = start_date - pd.DateOffset(weeks=range_weeks)
            
            # 筛选数据
            t_true = self.df[(self.df['date'] >= start_range) & (self.df['original_date'] <= end_date)]
            t_true = t_true[['date', 'true']].drop_duplicates().sort_values('date')
            
            # 筛选出start_date这一周的point数据，按不同method分组
            t_point = self.df[self.df['original_date'] == start_date]
            methods = t_point['method'].unique()
            
            # 清除旧图表
            self.figure.clear()
            ax = self.figure.add_subplot(111)
            
            # 绘制真实值
            ax.plot(t_true['date'], t_true['true'], label='True', marker='o', color='black')
            
            # 为每个method绘制预测值
            colors = ['red', 'blue', 'green', 'purple', 'orange']  # 可以根据需要添加更多颜色
            for i, method in enumerate(methods):
                method_data = t_point[t_point['method'] == method]
                method_data = method_data[['date', 'point', 'q_5', 'q_95']].drop_duplicates().sort_values('date')
                color = colors[i % len(colors)]
                ax.plot(method_data['date'], method_data['point'], 
                       label=f'Prediction ({method})', 
                       marker='x', 
                       color=color,
                       linestyle='--')
                ax.fill_between(method_data['date'],
                method_data['q_5'],
                method_data['q_95'],
                color=color,
                alpha=0.2,  # 透明度
                label=f'95% CI ({method})')

            
            # 设置图表属性
            ax.set_title(f'{start_date}')
            ax.set_xlabel('Date')
            ax.set_ylabel('Values')
            ax.legend()
            ax.grid()
            
            # 自动调整日期格式
            self.figure.autofmt_xdate()
            
            # 更新画布
            self.canvas.draw()
            
        except Exception as e:
            print(f"绘图时出错: {e}")

root = tk.Tk()
app = WeeklyDataVisualizer(root, df)
root.mainloop()